In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the data
mid = pd.read_csv("MID_series2.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())

#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())

#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)


       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    3.730000e+03    3.730000e+03     3.730000e+03     3.730000e+03   
mean     4.285301e-11    5.016836e-13     6.530565e-13     1.285517e-12   
std      1.163854e-11    2.656285e-13     3.544741e-13     9.366051e-13   
min      1.611400e-11    3.400000e-15     1.123000e-13     2.918000e-13   
25%      3.661275e-11    9.450000e-14     4.413000e-13     7.413000e-13   
50%      3.797520e-11    6.501500e-13     4.688000e-13     7.764500e-13   
75%      5.233625e-11    6.657000e-13     8.973500e-13     2.037325e-12   
max      1.027690e-10    8.547000e-13     1.837100e-12     1.066500e-11   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     3.730000e+03     3.730000e+03     3.730000e+03     3.729000e+03   
mean      1.122954e-12     3.995674e-12     3.749064e-14     8.563251e-13   
std       1.710698e-13     1.678985e-12     1.244857e-14     1.641974e-12   
min       5.4730

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/28 11:56:24.040,2022/11/28 11:56:24.290,2022/11/28 11:56:24.540,2022/11/28 11:56:24.790,2022/11/28 11:56:25.040,2022/11/28 11:56:25.290,2022/11/28 11:56:25.541,2022/11/28 11:56:26.041,NaN,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,2022/11/28 11:56:26.363,2022/11/28 11:56:26.613,2022/11/28 11:56:26.863,2022/11/28 11:56:27.113,2022/11/28 11:56:27.363,2022/11/28 11:56:27.613,2022/11/28 11:56:27.864,2022/11/28 11:56:28.364,NaN,2.323,2.323,2.323,2.323,2.323,2.323,2.323,2.323,NaN
4,2022/11/28 11:56:28.686,2022/11/28 11:56:28.936,2022/11/28 11:56:29.187,2022/11/28 11:56:29.437,2022/11/28 11:56:29.687,2022/11/28 11:56:29.937,2022/11/28 11:56:30.187,2022/11/28 11:56:30.688,NaN,4.646,4.646,4.647,4.647,4.647,4.647,4.646,4.647,NaN
5,2022/11/28 11:56:31.009,2022/11/28 11:56:31.259,2022/11/28 11:56:31.509,2022/11/28 11:56:31.760,2022/11/28 11:56:32.010,2022/11/28 11:56:32.260,2022/11/28 11:56:32.510,2022/11/28 11:56:33.011,NaN,6.969,6.969,6.969,6.970,6.970,6.970,6.969,6.970,NaN
6,2022/11/28 11:56:33.332,2022/11/28 11:56:33.582,2022/11/28 11:56:33.832,2022/11/28 11:56:34.082,2022/11/28 11:56:34.332,2022/11/28 11:56:34.583,2022/11/28 11:56:34.833,2022/11/28 11:56:35.083,2022/11/28 11:56:35.333,9.292,9.292,9.292,9.292,9.292,9.293,9.292,9.042,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3727,2022/11/28 14:20:42.362,2022/11/28 14:20:42.612,2022/11/28 14:20:42.862,2022/11/28 14:20:43.112,2022/11/28 14:20:43.362,2022/11/28 14:20:43.613,2022/11/28 14:20:43.863,2022/11/28 14:20:44.113,2022/11/28 14:20:44.363,8658.322,8658.322,8658.322,8658.322,8658.322,8658.323,8658.322,8658.072,NaN
3728,2022/11/28 14:20:44.684,2022/11/28 14:20:44.934,2022/11/28 14:20:45.185,2022/11/28 14:20:45.435,2022/11/28 14:20:45.685,2022/11/28 14:20:45.935,2022/11/28 14:20:46.185,2022/11/28 14:20:46.435,2022/11/28 14:20:46.686,8660.644,8660.644,8660.645,8660.645,8660.645,8660.645,8660.644,8660.394,NaN
3729,2022/11/28 14:20:47.007,2022/11/28 14:20:47.257,2022/11/28 14:20:47.507,2022/11/28 14:20:47.757,2022/11/28 14:20:48.007,2022/11/28 14:20:48.258,2022/11/28 14:20:48.508,2022/11/28 14:20:49.008,NaN,8662.967,8662.967,8662.967,8662.967,8662.967,8662.968,8662.967,8662.967,NaN
3730,2022/11/28 14:20:49.329,2022/11/28 14:20:49.579,2022/11/28 14:20:49.830,2022/11/28 14:20:50.080,2022/11/28 14:20:50.330,2022/11/28 14:20:50.580,2022/11/28 14:20:50.830,2022/11/28 14:20:51.080,2022/11/28 14:20:51.331,8665.289,8665.289,8665.290,8665.290,8665.290,8665.290,8665.289,8665.039,NaN


In [3]:
#reading the vaclogger file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,28/11/2022 11:55:12,8.0,-0.068828,1.340000e-09,5.190000e-10,4.996000e-09,3.873000e-07,0.000574,0.0,-109.203,56.565,NaN,NaN
2,NaN,28/11/2022 11:55:19,15.0,-0.069642,1.340000e-09,5.190000e-10,4.996000e-09,3.887000e-07,0.000581,0.0,-109.090,56.341,NaN,NaN
3,NaN,28/11/2022 11:55:26,22.0,-0.069517,1.350000e-09,5.200000e-10,4.996000e-09,3.887000e-07,0.000581,0.0,-108.978,56.453,NaN,NaN
4,NaN,28/11/2022 11:55:33,29.0,-0.069347,1.340000e-09,5.200000e-10,4.996000e-09,3.892000e-07,0.000545,0.0,-109.090,56.453,NaN,NaN
5,NaN,28/11/2022 11:55:40,36.0,-0.069076,1.340000e-09,5.190000e-10,4.996000e-09,3.892000e-07,0.000570,0.0,-109.090,56.453,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,7290.000000,7290.000000,7290.000000,7290.000000,7290.000000,7290.000000,7290.0,7290.0,7290.0,7290.000000,7290.000000,7290.000000,7290.000000,7290.000000,7290.000000,7290.000000,7290.000000,7.290000e+03,7.290000e+03
mean,4349.659344,342.825627,96.929342,967.322291,967.249871,967.072376,0.0,0.0,0.0,-0.064709,0.314837,37.854031,37.884731,37.869266,-0.015660,-0.027730,-0.021122,9.593584e-08,3.148367e-07
std,2512.823403,231.816728,17.491464,177.028744,177.476363,177.981995,0.0,0.0,0.0,0.142867,0.474261,6.996605,7.004829,7.006288,0.002024,0.001533,0.002165,1.240720e-07,4.742608e-07
min,1.179000,0.020000,0.080000,0.020000,0.000000,0.000000,0.0,0.0,0.0,-0.651300,0.006400,0.009700,0.009100,0.007600,-0.022500,-0.032800,-0.028000,0.000000e+00,6.400000e-09
25%,2173.430750,0.140000,100.080000,999.880000,1000.000000,1000.020000,0.0,0.0,0.0,-0.210800,0.029300,38.596825,38.628325,38.609700,-0.016700,-0.028600,-0.022000,2.180000e-08,2.930000e-08
50%,4349.640000,499.900000,100.100000,999.900000,1000.020000,1000.040000,0.0,0.0,0.0,0.021800,0.030300,39.385600,39.421950,39.405650,-0.014900,-0.027400,-0.020800,2.260000e-08,3.030000e-08
75%,6525.573500,499.900000,100.100000,999.920000,1000.020000,1000.040000,0.0,0.0,0.0,0.022500,0.836975,39.806400,39.841275,39.828475,-0.014300,-0.026900,-0.019500,2.115750e-07,8.369750e-07
max,8699.811000,499.920000,100.160000,999.940000,1000.040000,1000.060000,0.0,0.0,0.0,0.541000,3.525500,40.047800,40.081800,40.067000,-0.012500,-0.024300,-0.017800,6.513000e-07,3.525500e-06


### Data processing

In [5]:
#Filtering background signal - necessary only when calculating desorption yield

#Find the timestamp where emission was turned on
print("Result 1:",hv.loc[hv["Comment"]=="emission on"])
print("Result 2:",hv.loc[hv["Comment"]=="grid repelling"])

x=pd.Timestamp(2022, 11, 25, 11, 13, 0)
y=pd.Timestamp(2022, 11, 25, 11, 18, 5)

#then find the time value closest to it in the MID scan file
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#average the signal before this timestamp and subtract it from the Em-on part
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 8:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

#remove values under noise level from MID spectrum
mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][73:204]/h2_sens
print(p_h2.describe())
#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:73]/h2_sens
print(base_h2.describe())

Result 1:            Date          Time  Elapsed time  VMon1   VMon2   VMon3   VMon4  \
265  28/11/2022  12:01:10.189       315.047   0.16  100.12  999.64  999.62   
266  28/11/2022  12:01:11.379       316.237   0.14  100.12  999.64  999.62   
267  28/11/2022  12:01:12.604       317.462   0.14  100.12  999.66  999.62   
268  28/11/2022  12:01:13.826       318.684   0.14  100.12  999.66  999.64   

      VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
265  999.62    0.0    0.0  ...  1.3258  34.7288  34.7291  34.7281 -0.0204   
266  999.62    0.0    0.0  ...  1.3199  34.7473  34.7462  34.7444 -0.0204   
267  999.62    0.0    0.0  ...  1.3033  34.7569  34.7545  34.7535 -0.0204   
268  999.62    0.0    0.0  ...  1.2820  34.7721  34.7693  34.7684 -0.0204   

      IMon7  IMon8      Comment       hv_grid        em  
265 -0.0311 -0.025  emission on  3.606000e-07  0.000001  
266 -0.0311 -0.025  emission on  3.534000e-07  0.000001  
267 -0.0311 -0.025  emission on  3.4690

count    1.310000e+02
mean     2.268967e-09
std      1.019753e-09
min      7.184918e-10
25%      7.326256e-10
50%      2.878107e-09
75%      3.015689e-09
max      3.125055e-09
Name: mass 2, corr, dtype: float64
count    7.300000e+01
mean     7.284235e-10
std      6.546285e-12
min      7.163228e-10
25%      7.246749e-10
50%      7.271725e-10
75%      7.312852e-10
max      7.561162e-10
Name: mass 2, corr, dtype: float64


C:\Users\etiirine\AppData\Local\Temp\ipykernel_18972\3400534067.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan


In [6]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


### Plotting

In [7]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M12')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan, EGA out on Cu @ RT, dynamic vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 14,
        }
plt.text(-400, 2e-11, 'No em.', fontdict = font)
plt.text(325, 1e-10, 'Emission on', fontdict = font)
plt.text(1300, 2e-11, 'Grid repelling', fontdict = font)
plt.text(2305, 8e-11, 'Grid transparent', fontdict = font)
plt.text(3205, 2e-11, 'Grid repelling, EGA turned', fontdict = font)
plt.text(4255, 4.7e-12, 'Slight adsorption of the friction-desorbed gas', fontdict = font)

plt.yscale('log')

In [8]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("HiVolta current log, EGA out on Cu @ RT, dynamic vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 14,
        }
plt.text(400, 3e-6, 'Emission on', fontdict = font)
plt.text(1437, 7e-9, 'Grid repelling', fontdict = font)
plt.text(2400, 1.5e-6, 'Grid transparent', fontdict = font)
plt.text(3450, 8e-9, 'Grid repelling, EGA turned', fontdict = font)

plt.yscale('log')


In [9]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution, EGA out on Cu @ RT, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.show()

# Primary desorption calculation method
### Formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.


In [10]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = np.mean(polyfit(A,vaclog["T-CERNOX"]))
print(T)
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)

#Emission current
#ignore the inrush current values
hv["em"].where(hv["em"] <= 1.2e-6, np.nan, inplace=True)
I_em = np.nanmean(hv["em"].nlargest(100))
print(I_em)

#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


1.5405430593950968e-07
288.4343907175566
0.01992516581153798
1.1631249999999998e-06


In [11]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: 0.10608237897378979 

